In [1]:
import os
import sys
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
import ee
import rs
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
ee.Initialize()

In [2]:
# This function adds a time band to the image.
def createTimeBand(image):
    # Scale milliseconds by a large constant.
    return image.addBands(image.metadata('system:time_start').divide(1e18))

# This function adds a constant band to the image.


def createConstantBand(image):
    return ee.Image(1).addBands(image)


def addIndices(index_name_list):
    """Add vegetation indices to the image

    Args:
        index_name_list (list): list of indices to add to the image
    
    Returns:
        function: function to add indices to image (for use with map())

        The image must contain bands with literal names:
            'nir', 'red', 'green', 'blue', 'swir1', 'swir2'

        Vegetation indices may include: 
        NDVI = (nir - red) / (nir + red),
        MNDWI = (green - swir1) / (green + swir1),
        EVI = 2.5 * (nir - red) / (nir + 6 * red - 7 * blue + 1),
        GCVI = (nir / green) - 1,
        NBR1 = (nir - swir1) / (nir + swir1), # Also NDMI: https://www.usgs.gov/landsat-missions/normalized-difference-moisture-index
        NBR2 = (nir - swir2) / (nir + swir2),
        NDTI = (swir1 - swir2) / (swir1 + swir2), # for an example: https://www.mdpi.com/2072-4292/8/8/660

        Not included as of now:
        MODCRC = (swir1 - green) / (swir1 + green),
        SAVI = (nir - red) / (nir + red + 0.16),
        STI = swir1 / swir2,
        TVI = 60 * (nir - green) - 100 * (red - green)

    Examples:
        oli8 = prep_oli8_ic(ee_geometry) \
            .map(addIndices(['NDVI', 'MNDWI', 'EVI', 'GCVI']))
    """
    def addIndices_(image):
        if 'NDVI' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['nir', 'red']).rename('NDVI'))

        if 'GCVI' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['nir', 'green']).rename('GCVI'))

        if 'MNDWI' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['green', 'swir1']).rename('MNDWI'))
        
        if 'NBR1' in index_name_list: # Also NDMI: https://www.usgs.gov/landsat-missions/normalized-difference-moisture-index
            image = image.addBands(image.normalizedDifference(
                ['nir', 'swir1']).rename('NBR1'))

        if 'NBR2' in index_name_list:
            image = image.addBands(image.normalizedDifference(
                ['nir', 'swir2']).rename('NBR2'))
        
        if 'NDTI' in index_name_list: # for an example: https://www.mdpi.com/2072-4292/8/8/660
            image = image.addBands(image.normalizedDifference(
                ['swir1', 'swir2']).rename('NDTI'))
        
        if 'EVI' in index_name_list:
            image = image.addBands(image.expression(
                '2.5 * (im.nir - im.red) / (im.nir + 6 * im.red - 7 * im.blue + 1)', {'im': image}).rename('EVI'))

        if 'GCVI' in index_name_list:
            image = image.addBands(image.expression(
                '(im.nir / im.green) - 1', {'im': image}).rename('GCVI'))

        return image

    return addIndices_


In [3]:
ee_geometry = ee.FeatureCollection('users/gopalpenny/arkavathytanks/ArkavatiSubbasins')
ee_geometry = ee_geometry.filter(ee.Filter.eq('Subcatch', 'TG_Halli'))

tm5 = rs.prep_tm5_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
etm7 = rs.prep_etm7_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
oli8 = rs.prep_oli8_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')


# etm7.map(createTimeBand)

landsat = tm5.merge(etm7).merge(oli8) \
    .filter(ee.Filter.lt('CLOUD_COVER', 50)) \
    .map(addIndices(['GCVI']))

landsat_clouds = rs.prep_landsat_clouds(landsat)
landsat_clouds = landsat_clouds.map(lambda img: 
    img.updateMask(img.select('clouds_shadows').eq(0)))

In [4]:
m = geemap.Map()

m.addLayer(landsat_clouds, {'bands': ['nir','red','green'], 'min': 0, 'max': 30000}, 'landsat fcc')
# m.addLayer(landsat_clouds, {'bands': ['clouds_shadows'], 'min': 0, 'max': 1}, 'landsat_clouds')
m.centerObject(ee_geometry, 9)
m.addLayerControl()
m

Map(center=[13.173381307415593, 77.42822974527482], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
def addHarmonicBands(start_month):
    """_summary_

    Args:
        start_month_num (int): Month that serves as a zero reference for time

    Returns:
        func: Function that adds harmonic bands to an image for regression

        Returns a function can be used in ee.ImageCollection.map() to add harmonic
        bands to each image in the collection. See Wang et al (2019). 
        
        The harmonic bands are:

        * year_frac: Fraction of a year between 0 and 1, with 0 and 1 being the first day of start month
        * sin(3 * pi * t)
        * cos(3 * pi * t)
        * sin(6 * pi * t)
        * cos(6 * pi * t)


    Examples:
        oli8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_SR') \
            .map(addHarmonicBands(6))
        oli8
            .select(['constant', 'sin3pi', 'cos3pi', 'sin6pi', 'cos6pi', 'B5']) \
            .reduce(ee.Reducer.linearRegression(5, 1))
    """
    # (1e3 * 60 * 60 * 24 * 365) #/ 3.1536e10 # milliseconds in year
    start_month_num = start_month   # e.g., start_month = 6 --> start_date: 2014-0-01

    def addHarmonicBandsFunction(img):
        # get time as a fraction of a year

        img_date = ee.Date(img.get('system:time_start'))
        year = img_date.get('year')
        prev_year = img_date.get('month').subtract(start_month_num).lt(0)
        monsoon_year = ee.Number(year.subtract(prev_year))
        year_start_date = ee.Date.fromYMD(monsoon_year, start_month_num, 1)
        year_frac = img_date.millis().subtract(
            year_start_date.millis()).divide(3.1536e10)
        # img = img.set('year_frac', year_frac)
        img = img.set('year_frac', year_frac)
        year_frac = img.metadata('year_frac')
        time3pi = year_frac.multiply(9.42477796076938)
        time6pi = year_frac.multiply(18.84955592153876)

        # Add harmonic regression bands: sin(3 * pi * t), cos(3 * pi * t),
        # sin(6 * pi * t), cos(6 * pi * t), t = time in years since June 1
        img = img \
            .addBands(year_frac) \
            .addBands(ee.Image(1).rename('constant')) \
            .addBands(time3pi.cos().rename('cos3pi')) \
            .addBands(time3pi.sin().rename('sin3pi')) \
            .addBands(time6pi.cos().rename('cos6pi')) \
            .addBands(time6pi.sin().rename('sin6pi'))
        # img = img.addBands(
        #     img.select('year_frac')
        return img

    return addHarmonicBandsFunction


In [7]:
# harm_func = addRegressionBands(6)

landsat_harmonic_bands = landsat.map(addHarmonicBands(6))

# landsat_harmonic_bands.first().getInfo()

In [8]:

independent_bands = ee.List(['constant', 'sin3pi', 'cos3pi', 'sin6pi', 'cos6pi'])
dependent_bands = ee.List(['nir', 'swir1', 'swir2', 'GCVI'])

# ee.List([independent_bands, dependent_bands]).getInfo()
                     
landsat_harmonics_regression = landsat_harmonic_bands \
    .select(independent_bands.cat(dependent_bands)) \
    .reduce(ee.Reducer.linearRegression(independent_bands.length(), dependent_bands.length()))


harmonic_coeff = landsat_harmonics_regression \
    .select('coefficients') \
    .arrayFlatten(ee.List([independent_bands, dependent_bands]))

# harmonic_coeff.getInfo()


In [9]:
# Load training points from shapefile
train_pts_orig = gpd.read_file('../../spatial/unmod/training/training_pt_ids.shp')
train_pts_orig = train_pts_orig[['id', 'geometry']]
training_classes_orig = pd.read_csv('../../data/format/lulc_training_crops.csv')

# Load training classes from csv, subset to classes with > 10 points
training_classes_orig['count'] = training_classes_orig.groupby('crop').transform('count')
training_classes = training_classes_orig[training_classes_orig['count'] > 10]
training_classes['label'] = training_classes['crop'].rank(method='dense', ascending=False, ).astype(int) - 1

# Merge training points with training classes
train_pts = train_pts_orig.merge(training_classes, on='id', how='inner')

# Save training points labels to csv
train_pts_summary = train_pts[['crop', 'label', 'count']].drop_duplicates()
train_pts_summary.to_csv('../../data/format/lulc_training_labels.csv', index=False)

# Upload points to earth engine
class_pts = geemap.geopandas_to_ee(train_pts)
# class_pts.first().getInfo()
train_pts_summary

,crop,label,count
0,Palm,3,49
1,Fallow/ploughed,6,37
4,Legume/millet,5,327
15,Plantation,2,92
22,Vegetable,1,26
48,Orchard,4,32
197,grapes,0,19


In [10]:
type(class_pts)

ee.featurecollection.FeatureCollection

In [11]:
sample = harmonic_coeff.reduceRegions(class_pts, ee.Reducer.first(), 30)

# sample = harmonic_coeff.reduceRegions({
#     'collection': class_pts,
#     'reducer': ee.Reducer.first(),
#     'scale': 30
# })
sample.size().getInfo()

582

In [12]:
# Get training and testing splits
sample = sample.randomColumn()
training_sample = sample.filter('random <= 0.8')
testing_sample = sample.filter('random > 0.8')

In [13]:
# Train a random forest classifier with n trees
n = 10
rf_classifier = ee.Classifier.smileRandomForest(n) \
    .train(training_sample, 'label', harmonic_coeff.bandNames())

In [126]:

# # Get information about the trained classifier.
# print('Results of trained classifier', rf_classifier.explain())

# Get a confusion matrix and overall accuracy for the training sample.
trainAccuracy = rf_classifier.confusionMatrix()
# trainAccuracy.getInfo()
# print('Training error matrix', trainAccuracy)
# print('Training overall accuracy', trainAccuracy.accuracy())


In [14]:
# Classify the reflectance image from the trained classifier.
imgClassified = harmonic_coeff.classify(rf_classifier)


In [17]:
Map = geemap.Map()
classVis = {
  'min': 0,
  'max': 6,
  'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4'] # , 'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']
}
Map.centerObject(class_pts, 9)
# Map.addLayer(img, {bands: ['B11', 'B8', 'B3'], min: 100, max: 3500}, 'img');
# Map.addLayer(lc, classVis, 'lc');
Map.addLayer(landsat_clouds, {'bands': ['nir','red','green'], 'min': 0, 'max': 30000}, 'landsat fcc')
Map.addLayer(imgClassified, classVis, 'Classified')
Map.addLayerControl()
# Map.addLayer(roi, {color: 'white'}, 'ROI', false, 0.5);
# Map.addLayer(trainingSample, {color: 'black'}, 'Training sample', false);
# Map.addLayer(validationSample, {color: 'white'}, 'Validation sample', false);
Map

Map(center=[13.192831769553079, 77.40759098509344], controls=(WidgetControl(options=['position', 'transparent_…

In [136]:
m1

Map(center=[13.192831769553079, 77.40759098509344], controls=(WidgetControl(options=['position', 'transparent_…

In [130]:
imgClassified.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}